In [ ]:
import ee
import geemap

try:
    ee.Initialize()
    print("Google Earth Engine에 성공적으로 연결되었습니다.")
except Exception as e:
    ee.Authenticate(auth_mode='notebook')
    print(f"Earth Engine 초기화에 실패했습니다. 'earthengine authenticate'를 실행했는지 확인하세요. 오류: {e}")
    print("스크립트를 중단합니다.")
    exit()



In [ ]:
import ee
import time
import random
import numpy as np
import rasterio
from rasterio.transform import from_bounds
import os
from PIL import Image
import requests
import io
import math

# --- 0. GEE 인증 및 초기화 (필수) ---
# 스크립트 실행 시 GEE에 연결하기 위해 필요합니다.
try:
    ee.Initialize()
    print("Google Earth Engine에 성공적으로 연결되었습니다.")
except Exception as e:
    print(f"Earth Engine 초기화에 실패했습니다. 'earthengine authenticate'를 실행했는지 확인하세요. 오류: {e}")
    print("스크립트를 중단합니다.")
    exit()

# --- 1. 사용자 설정 (필수 수정) ---

# 1. GEE Asset 경로: (업로드한 CSV 파일의 Asset ID로 변경하세요)
ASSET_PATH = 'projects/gen-lang-client-0048128925/assets/gee_weather_ground_truth_balanced' 

# 2. 샘플링할 총 개수 (G)
G_SAMPLES = 50000  # (5000개 이상으로 설정)

# 3. 배치 크기 (BATCH_SIZE): GEE에서 한 번에 가져올 피처 개수
# (너무 크면 오류, 너무 작으면 비효율적. 500 ~ 1000 권장)
BATCH_SIZE = 500

# 스케일 풀
SCALE_POOL = [32000, 64000, 128000, 256000, 512000, 1024000]  # 미터 단위

# 4. 데이터 해상도 (Scale)
#    (칩의 픽셀 크기, 미터(m) 단위. 500m MODIS 데이터 기준)
PIXEL_SCALE_METERS = 500

# 5. 로컬 저장 설정
RGB_FOLDER = './rgb3'
IR_FOLDER = './ir3'

# 폴더 생성
os.makedirs(RGB_FOLDER, exist_ok=True)
os.makedirs(IR_FOLDER, exist_ok=True)

# 6. MODIS 촬영 시간 (local time 기준, 시간 단위)
TERRA_DAY_LOCAL_HOUR = 10.5    # Terra 주간: 오전 10:30
AQUA_DAY_LOCAL_HOUR = 13.5     # Aqua 주간: 오후 1:30
TERRA_NIGHT_LOCAL_HOUR = 22.5  # Terra 야간: 오후 10:30
AQUA_NIGHT_LOCAL_HOUR = 1.5    # Aqua 야간: 오전 1:30

# 7. 타겟 시간대 정의
TARGET_LOCAL_HOURS = [10, 11, 13, 14, 22, 23, 1, 2]
DAY_HOURS = [10, 11, 13, 14]    # 주간
NIGHT_HOURS = [22, 23, 1, 2]    # 야간

# 8. 이미지 유효성 검사 임계값
EMPTY_PIXEL_THRESHOLD = 0.50

# --- 2. GEE 데이터셋 정의 ---

# 1) RGB + IR (MODIS 일별 데이터 사용)
terra = ee.ImageCollection('MODIS/061/MOD09GA')    # Terra RGB (500m) - 주간만
aqua = ee.ImageCollection('MODIS/061/MYD09GA')     # Aqua RGB (500m) - 주간만
terra_lst = ee.ImageCollection('MODIS/061/MOD11A1')  # Terra LST (1km) - 주/야간
aqua_lst = ee.ImageCollection('MODIS/061/MYD11A1')   # Aqua LST (1km) - 주/야간

# 2) 수치 기상 정보 (ERA5 시간별 데이터 사용)
era5_collection = ee.ImageCollection('ECMWF/ERA5/HOURLY')

# --- 3. 샘플링 및 작업 시작 ---

print(f"Asset '{ASSET_PATH}'에서 피처를 로드합니다...")
# 1. Asset 로드
fc = ee.FeatureCollection(ASSET_PATH)

# 2. G개 랜덤 샘플링 (서버 사이드) 및 리스트 변환
print(f"총 {G_SAMPLES}개의 랜덤 샘플을 서버 사이드에서 정렬합니다...")
fc_sample = fc.randomColumn().sort('random').limit(G_SAMPLES)

print(f"서버 사이드 피처 컬렉션을 리스트로 변환합니다 (시간이 걸릴 수 있음)...")
try:
    fc_list = fc_sample.toList(G_SAMPLES)
    print(f"서버 사이드 리스트 생성 완료.")
except Exception as e:
    print(f"오류: GEE 서버에서 {G_SAMPLES}개 샘플 리스트 생성 중 실패. {e}")
    print("G_SAMPLES 개수를 줄이거나 Asset을 확인하세요. 스크립트를 중단합니다.")
    exit()


num_batches = math.ceil(G_SAMPLES / BATCH_SIZE)
print(f"--- 총 {G_SAMPLES}개 작업을 {BATCH_SIZE}개씩 {num_batches}개 배치로 나누어 시작합니다 ---")

total_success_count = 0

# 3. Python 루프를 돌며 G개의 작업을 배치로 나눠서 시작
for batch_num in range(num_batches):
    batch_start_index = batch_num * BATCH_SIZE
    batch_end_index = min((batch_num + 1) * BATCH_SIZE, G_SAMPLES)
    
    print(f"\n--- 배치 {batch_num + 1}/{num_batches} (샘플 {batch_start_index + 1} ~ {batch_end_index}) 처리 시작 ---")

    # 4. GEE에서 현재 배치에 해당하는 피처 리스트 조각(slice)을 요청
    try:
        current_batch_list = ee.List(fc_list.slice(batch_start_index, batch_end_index))
        features = current_batch_list.getInfo() # <-- 이 부분이 핵심. BATCH_SIZE(500개) 만큼만 getInfo() 호출
        print(f"성공: {len(features)}개의 피처 정보를 클라이언트로 가져왔습니다.")
    except Exception as e:
        print(f"오류: 배치 {batch_num + 1}의 피처 정보를 가져오는 데 실패했습니다. ({e})")
        print("이 배치를 건너뛰고 다음 배치를 시도합니다.")
        continue

    # 5. 가져온 배치 피처를 하나씩 처리 (기존 로직과 거의 동일)
    for i_in_batch, feature_dict in enumerate(features):
        
        global_index = batch_start_index + i_in_batch # 전체 샘플 중 현재 인덱스
        
        # 6. 각 피처(샘플) 정보 추출
        try:
            properties = feature_dict['properties']
            
            # Asset에서 읽어온 정보
            time_str = properties['time'] 
            lon = properties['lon']
            lat = properties['lat']
            coco = properties.get('coco', 'unknown')
            local_hour = properties['local_hour']  # 현지 시각 (시간 단위)
            edxy_id = properties.get('id', f'EDXY{global_index}')  # ID 필드 (없으면 EDXY{i} 사용)
            
            # BOX_SIZE_METERS 랜덤 선택
            BOX_SIZE_METERS = random.choice(SCALE_POOL)
            
            # 랜덤 오프셋 생성 (x: [0.1, 0.9], y: [0.1, 0.9])
            offset_x = random.uniform(0.1, 0.9)
            offset_y = random.uniform(0.1, 0.9)
            
            # 오프셋을 미터 단위로 변환 (중심에서의 거리)
            offset_x_meters = (offset_x - 0.5) * BOX_SIZE_METERS
            offset_y_meters = (offset_y - 0.5) * BOX_SIZE_METERS
            
            # 위도/경도 오프셋 계산 (대략적인 변환: 1도 ≈ 111km)
            offset_lon = offset_x_meters / (111000 * np.cos(np.radians(lat)))
            offset_lat = offset_y_meters / 111000
            
            # 새로운 중심점 계산
            center_lon = lon + offset_lon
            center_lat = lat + offset_lat
            
            coords = [center_lon, center_lat]
            
            # 경계 좌표 계산 (박스의 네 모서리)
            half_box_meters = BOX_SIZE_METERS / 2
            lat_offset = half_box_meters / 111000
            lon_offset = half_box_meters / (111000 * np.cos(np.radians(center_lat)))
            
            lat_start = center_lat - lat_offset  # 남쪽 (하단)
            lat_end = center_lat + lat_offset    # 북쪽 (상단)
            lng_start = center_lon - lon_offset  # 서쪽 (좌측)
            lng_end = center_lon + lon_offset    # 동쪽 (우측)
            
            # 날짜 형식 추출 (YYMMDD)
            date_str = time_str.split()[0]  # 'YYYY-MM-DD'
            hour_str = time_str.split()[1].split(':')[0]  # 'HH'
            date_formatted = date_str[2:].replace('-', '') + hour_str  # 'YYMMDDHH'

        except KeyError as e:
            print(f"경고: {global_index + 1}번째 피처에 {e} 속성이 없습니다. Asset 컬럼 이름('time', 'lat', 'lon', 'local_hour')을 확인하세요. 건너뜁니다.")
            continue
        except Exception as e:
            print(f"경고: {global_index + 1}번째 피처 속성 처리 중 오류: {e}. 건너뜁니다.")
            continue

        # 7. 주간/야간 판단 및 Terra vs Aqua 선택
        is_daytime = local_hour in DAY_HOURS
        is_nighttime = local_hour in NIGHT_HOURS
        
        if is_daytime:
            # 주간: Terra (10:30) vs Aqua (13:30)
            terra_diff = abs(local_hour - TERRA_DAY_LOCAL_HOUR)
            aqua_diff = abs(local_hour - AQUA_DAY_LOCAL_HOUR)
            
            if terra_diff < aqua_diff:
                selected_satellite = 'terra'
                modis_rgb_collection = terra
                modis_lst_collection = terra_lst
            else:
                selected_satellite = 'aqua'
                modis_rgb_collection = aqua
                modis_lst_collection = aqua_lst
                
        elif is_nighttime:
            # 야간: Terra (22:30) vs Aqua (01:30)
            terra_diff = min(abs(local_hour - TERRA_NIGHT_LOCAL_HOUR), 
                             abs(local_hour - TERRA_NIGHT_LOCAL_HOUR + 24),
                             abs(local_hour - TERRA_NIGHT_LOCAL_HOUR - 24))
            aqua_diff = min(abs(local_hour - AQUA_NIGHT_LOCAL_HOUR),
                            abs(local_hour - AQUA_NIGHT_LOCAL_HOUR + 24),
                            abs(local_hour - AQUA_NIGHT_LOCAL_HOUR - 24))
            
            if terra_diff < aqua_diff:
                selected_satellite = 'terra'
                modis_rgb_collection = None # 야간에는 RGB 없음
                modis_lst_collection = terra_lst
            else:
                selected_satellite = 'aqua'
                modis_rgb_collection = None # 야간에는 RGB 없음
                modis_lst_collection = aqua_lst
        else:
            print(f"경고: {global_index + 1}번째 피처 (local_hour={local_hour})는 타겟 시간대가 아닙니다. 건너뜁니다.")
            continue

        # --- 8. GEE 처리 및 다운로드 (더 견고한 try-except로 감싸기) ---
        # 이 try 블록은 GEE 이미지 처리(mean, select) 또는 
        # getThumbURL, requests.get 에서 발생하는 모든 오류를 잡아냅니다.
        try:
            # ISO 8601 형식으로 변환
            time_str_iso = time_str.replace(' ', 'T')
            time_start = ee.Date(time_str_iso)
                
            point = ee.Geometry.Point(coords) 
            region_box = point.buffer(BOX_SIZE_METERS / 2).bounds()

            # MODIS (일별 데이터) : 해당 날짜의 자정 ~ 다음 날 자정
            modis_time_start = time_start.update(hour=0, minute=0, second=0)
            modis_time_end = modis_time_start.advance(1, 'day')

            # 파일 이름 생성
            time_period = 'day' if is_daytime else 'night'
            file_name_base = f"{edxy_id}_{selected_satellite}_{time_period}_coco{coco}_{date_formatted}_lh{int(local_hour):02d}_{lat_start:.6f}_{lng_start:.6f}_{lat_end:.6f}_{lng_end:.6f}"

            rgb_saved = False
            ir_saved = False

            # 9. RGB 이미지 처리 (주간만)
            if is_daytime and modis_rgb_collection is not None:
                modis_rgb_filtered = modis_rgb_collection \
                                    .filterDate(modis_time_start, modis_time_end) \
                                    .filterBounds(point)
                
                # mean()은 빈 컬렉션에서 오류를 내지 않고, 밴드가 없는 이미지를 반환
                modis_rgb = modis_rgb_filtered.mean()
                
                # select()에서 밴드가 없으면 오류 발생 -> except에서 잡힘
                rgb = modis_rgb.select(
                    ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
                    ['RGB_B1_Red', 'RGB_B4_Green', 'RGB_B3_Blue']
                ).multiply(0.0001)

                rgb_vis = rgb.visualize(
                    bands=['RGB_B1_Red', 'RGB_B4_Green', 'RGB_B3_Blue'],
                    min=0,
                    max=1
                )
                
                rgb_url = rgb_vis.getThumbURL({
                    'region': region_box.getInfo()['coordinates'], # getInfo()가 여기서도 사용됨
                    'dimensions': [128, 128],
                    'format': 'png'
                })
                
                # URL에서 이미지 다운로드
                response = requests.get(rgb_url)
                if response.status_code == 200:
                    img = Image.open(io.BytesIO(response.content))
                    img_array = np.array(img)
                    
                    if len(img_array.shape) == 3:
                        black_pixels = np.all(img_array == 0, axis=2)
                        empty_ratio = np.sum(black_pixels) / (img_array.shape[0] * img_array.shape[1])
                    else:
                        empty_ratio = np.sum(img_array == 0) / img_array.size
                    
                    if empty_ratio > EMPTY_PIXEL_THRESHOLD:
                        print(f"  [경고] {global_index + 1}: RGB 이미지의 {empty_ratio*100:.1f}%가 빈 픽셀입니다. 건너뜁니다.")
                        continue
                    
                    rgb_path = os.path.join(RGB_FOLDER, f"{file_name_base}.png")
                    with open(rgb_path, 'wb') as f:
                        f.write(response.content)
                    rgb_saved = True
                else:
                    print(f"  [오류] {global_index + 1}: RGB 다운로드 실패 (HTTP {response.status_code})")
                    continue
            
            # 10. IR 이미지 처리 (주/야간 모두)
            modis_lst_filtered = modis_lst_collection \
                                .filterDate(modis_time_start, modis_time_end) \
                                .filterBounds(point)
            
            modis_ir = modis_lst_filtered.mean()
            
            if is_daytime:
                # 주간 LST
                ir = modis_ir.select('LST_Day_1km') \
                             .multiply(0.02) \
                             .subtract(273.15) \
                             .rename('IR_LST_Celsius')
            else:
                # 야간 LST
                ir = modis_ir.select('LST_Night_1km') \
                             .multiply(0.02) \
                             .subtract(273.15) \
                             .rename('IR_LST_Celsius')
            
            ir_vis = ir.visualize(
                min=-20,
                max=40,
                palette=['blue', 'cyan', 'green', 'yellow', 'red']
            )
            
            ir_url = ir_vis.getThumbURL({
                'region': region_box.getInfo()['coordinates'], # getInfo()
                'dimensions': [128, 128],
                'format': 'png'
            })
            
            response = requests.get(ir_url)
            if response.status_code == 200:
                img = Image.open(io.BytesIO(response.content))
                img_array = np.array(img)
                
                if len(img_array.shape) == 3:
                    black_pixels = np.all(img_array == 0, axis=2)
                    empty_ratio = np.sum(black_pixels) / (img_array.shape[0] * img_array.shape[1])
                else:
                    empty_ratio = np.sum(img_array == 0) / img_array.size
                
                # if empty_ratio > EMPTY_PIXEL_THRESHOLD:
                #     print(f"  [경고] {global_index + 1}: IR 이미지의 {empty_ratio*100:.1f}%가 빈 픽셀입니다. 건너뜁니다.")
                #     continue
                
                ir_path = os.path.join(IR_FOLDER, f"{file_name_base}.png")
                with open(ir_path, 'wb') as f:
                    f.write(response.content)
                ir_saved = True
            else:
                print(f"  [오류] {global_index + 1}: IR 다운로드 실패 (HTTP {response.status_code})")
                continue

            # 11. 최종 성공 로그
            if (is_daytime and rgb_saved and ir_saved) or (is_nighttime and ir_saved):
                print(f"  [OK] {global_index + 1}: {file_name_base}.png 저장 완료 (BOX_SIZE={BOX_SIZE_METERS}m)")
                total_success_count += 1
            elif is_daytime and not rgb_saved:
                 print(f"  [경고] {global_index + 1}: IR은 저장했으나 RGB 저장에 실패했습니다.")
            
        except Exception as e:
            # GEE 처리 오류 (e.g., 'Image.select: Input image does not have a band named...')
            # 또는 getThumbURL, requests.get 관련 오류 모두 여기서 처리
            print(f"  [오류] {global_index + 1}번째 피처 처리/다운로드 중 예외 발생: {e}. 건너뜁니다.")
            continue

        time.sleep(0.1) # API 제한을 피하기 위한 딜레이 (0.5초에서 줄여도 배치로 인해 괜찮을 수 있음)

print(f"\n--- 모든 배치 작업이 완료되었습니다. ---")
print(f"총 {G_SAMPLES}개 요청 중 {total_success_count}개의 샘플이 성공적으로 처리/저장되었습니다.")
print(f"RGB 이미지: '{RGB_FOLDER}' 폴더")
print(f"IR 이미지: '{IR_FOLDER}' 폴더")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

file = "../data/weather_chip_00000_Terra_night_coco8_lh23_time2023-03-30_21-00-00.tif"

# GeoTIFF 파일 열기
with rasterio.open(file) as src:
    # 메타데이터 출력
    print("=== 파일 정보 ===")
    print(f"밴드 수: {src.count}")
    print(f"크기: {src.width} x {src.height}")
    print(f"CRS: {src.crs}")
    print(f"Transform: {src.transform}")
    print(f"Bounds: {src.bounds}")
    print(f"\n밴드 이름: {src.descriptions}")
    
    # 메타데이터 태그 출력
    print(f"\n=== 메타데이터 ===")
    tags = src.tags()
    for key, value in tags.items():
        print(f"{key}: {value}")
    
    # 모든 밴드 읽기
    data = src.read()
    
    print(f"\n=== 데이터 형태 ===")
    print(f"Shape: {data.shape}")
    print(f"Data type: {data.dtype}")

# 시각화
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

band_names = [
    'RGB_B1_Red', 'RGB_B4_Green', 'RGB_B3_Blue', 'IR_LST_Celsius',
    'ERA5_Precip_mm', 'ERA5_Wind_U', 'ERA5_Wind_V', 'ERA5_MSLP_hPa'
]

for i in range(min(8, data.shape[0])):
    ax = axes[i]
    im = ax.imshow(data[i], cmap='viridis')
    ax.set_title(f'Band {i+1}: {band_names[i] if i < len(band_names) else "Unknown"}')
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046)

plt.tight_layout()
plt.show()

# RGB 합성 이미지 (밴드 1,2,3)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
rgb = np.dstack([data[0], data[1], data[2]])
# 값 범위 조정 (0-1)
rgb = np.clip(rgb, 0, 1)
ax.imshow(rgb)
ax.set_title('RGB Composite')
ax.axis('off')
plt.show()

In [ ]:
import folium
import numpy as np
from PIL import Image
import io
import base64
import matplotlib.cm as cm
import glob
import os
import re
import random

# ./rgb 폴더와 ./ir 폴더의 모든 .png 파일 찾기
rgb_folder = "./rgb2"
ir_folder = "./ir2"

rgb_files = glob.glob(os.path.join(rgb_folder, "*.png"))
ir_files = glob.glob(os.path.join(ir_folder, "*.png"))

# 랜덤으로 CNT개만 선택
CNT = 10

print(f"총 {len(rgb_files)}개의 RGB 파일과 {len(ir_files)}개의 IR 파일을 찾았습니다.")

# RGB와 IR 파일에서 랜덤으로 CNT개씩 선택
if len(rgb_files) > CNT:
    rgb_files = random.sample(rgb_files, CNT)
    print(f"RGB 파일 중 랜덤으로 {CNT}개 선택했습니다.")

if len(ir_files) > CNT:
    ir_files = random.sample(ir_files, CNT)
    print(f"IR 파일 중 랜덤으로 {CNT}개 선택했습니다.")

# 파일명에서 좌표 추출 함수
def extract_bounds_from_filename(filename):
    """
    파일명 형식: EDXY0_terra_day_coco19_230414_lh00_{lat_start}_{lng_start}_{lat_end}_{lng_end}.png
    """
    basename = os.path.basename(filename)
    parts = basename.replace('.png', '').split('_')
    
    try:
        # 마지막 4개가 lat_start, lng_start, lat_end, lng_end
        lat_start = float(parts[-4])
        lng_start = float(parts[-3])
        lat_end = float(parts[-2])
        lng_end = float(parts[-1])
        return lat_start, lng_start, lat_end, lng_end
    except:
        return None

def is_day_image(filename):
    """파일명에 '_day_'가 포함되어 있는지 확인"""
    return '_day_' in filename

# 모든 파일의 중심 좌표 계산 (지도 중심 설정용)
all_lats = []
all_lngs = []

for file_path in rgb_files + ir_files:
    bounds = extract_bounds_from_filename(file_path)
    if bounds:
        lat_start, lng_start, lat_end, lng_end = bounds
        center_lat = (lat_start + lat_end) / 2
        center_lng = (lng_start + lng_end) / 2
        all_lats.append(center_lat)
        all_lngs.append(center_lng)

if len(all_lats) == 0:
    print("파일에서 좌표를 추출할 수 없습니다!")
    m = None
else:
    # 모든 파일의 중심점을 기준으로 지도 생성
    center_lat = sum(all_lats) / len(all_lats)
    center_lng = sum(all_lngs) / len(all_lngs)
    
    # folium 지도 생성
    m = folium.Map(location=[center_lat, center_lng], zoom_start=6, tiles='OpenStreetMap')
    
    # RGB 파일 처리 (주간만)
    rgb_count = 0
    for idx, file_path in enumerate(rgb_files):
        file_name = os.path.basename(file_path)
        
        # 주간 이미지만 처리
        if not is_day_image(file_name):
            continue
            
        print(f"RGB 처리 중 ({idx+1}/{len(rgb_files)}): {file_name}")
        
        bounds = extract_bounds_from_filename(file_path)
        if not bounds:
            print(f"  경고: 좌표 추출 실패 - {file_name}")
            continue
            
        lat_start, lng_start, lat_end, lng_end = bounds
        
        try:
            # PNG 파일을 base64로 인코딩
            with open(file_path, 'rb') as f:
                img_data = f.read()
            img_b64 = base64.b64encode(img_data).decode()
            
            # RGB 레이어 추가 (기본으로 표시)
            folium.raster_layers.ImageOverlay(
                image=f'data:image/png;base64,{img_b64}',
                bounds=[[lat_start, lng_start], [lat_end, lng_end]],
                name=f'RGB - {file_name}',
                opacity=0.8,
                interactive=True,
                cross_origin=False,
                show=True  # 기본으로 표시
            ).add_to(m)
            
            # 경계 표시
            folium.Rectangle(
                bounds=[[lat_start, lng_start], [lat_end, lng_end]],
                color='blue',
                fill=False,
                weight=2,
                popup=f'RGB: {file_name}',
                tooltip=f'RGB: {file_name}'
            ).add_to(m)
            
            rgb_count += 1
            
        except Exception as e:
            print(f"  오류 ({file_name}): {e}")
            continue
    
    # IR 파일 처리 (모든 시간대)
    ir_count = 0
    for idx, file_path in enumerate(ir_files):
        file_name = os.path.basename(file_path)
        print(f"IR 처리 중 ({idx+1}/{len(ir_files)}): {file_name}")
        
        bounds = extract_bounds_from_filename(file_path)
        if not bounds:
            print(f"  경고: 좌표 추출 실패 - {file_name}")
            continue
            
        lat_start, lng_start, lat_end, lng_end = bounds
        
        try:
            # PNG 파일을 base64로 인코딩
            with open(file_path, 'rb') as f:
                img_data = f.read()
            img_b64 = base64.b64encode(img_data).decode()
            
            # IR 레이어 추가 (기본으로 표시)
            folium.raster_layers.ImageOverlay(
                image=f'data:image/png;base64,{img_b64}',
                bounds=[[lat_start, lng_start], [lat_end, lng_end]],
                name=f'IR - {file_name}',
                opacity=0.7,
                interactive=True,
                cross_origin=False,
                show=True  # 기본으로 표시
            ).add_to(m)
            
            # 경계 표시
            folium.Rectangle(
                bounds=[[lat_start, lng_start], [lat_end, lng_end]],
                color='red',
                fill=False,
                weight=1,
                popup=f'IR: {file_name}',
                tooltip=f'IR: {file_name}'
            ).add_to(m)
            
            ir_count += 1
            
        except Exception as e:
            print(f"  오류 ({file_name}): {e}")
            continue
    
    # 레이어 컨트롤 추가
    folium.LayerControl(collapsed=False).add_to(m)
    
    print(f"\n완료! 지도에 RGB {rgb_count}개, IR {ir_count}개 레이어를 추가했습니다.")
    print("모든 레이어가 기본으로 표시되며, 우측 상단의 레이어 컨트롤에서 ON/OFF 할 수 있습니다.")

# 지도 표시
m
